# 动手学深度学习 6.4节: 多输入输出通道 示例代码

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

多输入通道

In [5]:

X = torch.tensor(
	[
		[
			[0.0, 1.0, 2.0],
			[3.0, 4.0, 5.0],
			[6.0, 7.0, 8.0]
		],
		[
			[1.0, 2.0, 3.0],
			[4.0, 5.0, 6.0],
			[7.0, 8.0, 9.0]
		]
	]
)

K = torch.tensor(
	[
		[
			[0.0, 1.0],
			[2.0, 3.0]
		],
		[
			[1.0, 2.0],
			[3.0, 4.0]
		]
	]
)

In [6]:
def corr2d_multi_in(X, K):
	return sum(d2l.corr2d(x, k) for (x, k) in zip(X, K))

print(corr2d_multi_in(X, K))

tensor([[ 56.,  72.],
        [104., 120.]])


多输出通道

In [7]:
K = torch.stack((K, K + 1, K + 2), 0)

In [12]:
def corr2d_multi_in_out(X, K):
	return torch.stack([corr2d_multi_in(X, k) for k in K], 0)	# type: ignore


corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [11]:
def my_corr2d_multi_in_out(X:torch.Tensor, K:torch.Tensor):
	"""
	K.shape = (输出通道, 输入通道, 高, 宽)
	X.shape = (输入通道, 高, 宽)
	"""
	Y = torch.zeros([K.shape[0], X.shape[1] - K.shape[2] + 1, X.shape[2] - K.shape[3] + 1])

	# temp.shape = (输入通道, 输出通道, 输出高, 输出宽)
	temp = torch.zeros([K.shape[1], K.shape[0], X.shape[1] - K.shape[2] + 1, X.shape[2] - K.shape[3] + 1])

	# i表示输入通道
	for i in range(K.shape[1]):
		# j表示输出通道
		for j in range(K.shape[0]):
			temp[i, j] = d2l.corr2d(X[i], K[j,i,:,:])
	
	Y = torch.sum(temp, dim=0)
	return Y
		
	
my_corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1 X 1 卷积层

In [36]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [37]:
def corr2d_multi_in_out_1x1(X:torch.Tensor, K:torch.Tensor):
	c_i, n_h, n_w = X.shape
	c_o = K.shape[0]
	X = X.reshape([c_i, n_h*n_w])
	K = K.reshape([c_o, c_i])
	# 1*1卷积的本质就是MLP, 这里直接做线性运算
	Y = torch.matmul(K, X)
	return Y

In [38]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print(Y1 - Y2.reshape(Y1.shape))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]])
